In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from finta import TA


### Step 1: Use Alpaca to get SPY data.

In [2]:
import alpaca_trade_api as tradeapi

alpaca_api_key = 'PKXUNEDZ7VLE1EYMR65D'
alpaca_secret_key = '9o20jTOiT6jwikAbN2gNTrtku09lqm8bX02k2WBs'
alpaca_api_base_url = "https://paper-api.alpaca.markets"

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

# Set the ticker
ticker = "SPY"

# Set timeframe to "1Day"
timeframe = "1Day"

# Set start and end datetimes of 5 years
start_date = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2024-01-02", tz="America/New_York").isoformat()

# Get 5 years worth of historical data for SPY
ticker_data = api.get_bars(
    ticker,
    timeframe,
    start=start_date,
    end=end_date
).df

ticker_data.head()

,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-01-04 05:00:00+00:00,368.97,375.45,364.82,623066,375.31,111804895,369.329637
2021-01-05 05:00:00+00:00,371.40,372.50,368.05,338929,368.10,67456696,370.404421
2021-01-06 05:00:00+00:00,373.41,376.98,369.12,575346,369.71,109080743,373.804762
2021-01-07 05:00:00+00:00,379.13,379.90,375.91,366624,376.10,70220837,378.266600
2021-01-08 05:00:00+00:00,381.29,381.49,377.10,391374,380.59,74338260,380.152366


In [3]:
# Filter the date index and close columns

spy_df = ticker_data.loc[:, ["open", "high", "low", "close","volume" ]]

# Use the pct_change function to generate returns from close prices
spy_df["Actual Returns"] = spy_df["close"].pct_change()

# Drop all NaN values from the DataFrame
spy_df = spy_df.dropna()

# Review the DataFrame
display(spy_df.head())
display(spy_df.tail())

,open,high,low,close,volume,Actual Returns
timestamp,,,,,,
2021-01-05 05:00:00+00:00,368.10,372.50,368.0500,371.40,67456696,0.006586
2021-01-06 05:00:00+00:00,369.71,376.98,369.1200,373.41,109080743,0.005412
2021-01-07 05:00:00+00:00,376.10,379.90,375.9100,379.13,70220837,0.015318
2021-01-08 05:00:00+00:00,380.59,381.49,377.1000,381.29,74338260,0.005697
2021-01-11 05:00:00+00:00,377.85,380.58,377.7189,378.70,52148184,-0.006793


,open,high,low,close,volume,Actual Returns
timestamp,,,,,,
2023-12-26 05:00:00+00:00,474.07,476.58,473.99,475.65,55386952,0.004223
2023-12-27 05:00:00+00:00,475.44,476.66,474.89,476.51,68000811,0.001808
2023-12-28 05:00:00+00:00,476.88,477.55,476.26,476.69,77158117,0.000378
2023-12-29 05:00:00+00:00,476.49,477.03,473.30,475.31,122268009,-0.002895
2024-01-02 05:00:00+00:00,472.16,473.67,470.49,472.65,123007793,-0.005596


## Step 2: Generate trading signals using short- and long-window indicator values. 

In [4]:
#Recreate Trading Algo 

# Create a signals_df DataFrame

signals_df = spy_df.copy()

# Set the short window and long windows
short_window = 20
long_window = 30

# Add the HMA technical indicators for the short and long windows
signals_df["Short"] = TA.EMV(signals_df, short_window)
signals_df["Long"] = TA.EMV(signals_df, long_window)

# Review the DataFrame
signals_df.iloc[95:105, :]

,open,high,low,close,volume,Actual Returns,Short,Long
timestamp,,,,,,,,
2021-05-21 04:00:00+00:00,416.87,418.20,414.4500,414.98,77590498,-0.000409,0.001745,0.009461
2021-05-24 04:00:00+00:00,417.34,420.32,417.0800,419.17,52606963,0.010097,0.006436,0.013218
2021-05-25 04:00:00+00:00,420.33,420.71,417.6200,418.33,58365218,-0.002004,0.008189,0.012250
2021-05-26 04:00:00+00:00,418.87,419.61,417.7600,419.04,43861538,0.001697,0.005685,0.011426
2021-05-27 04:00:00+00:00,420.17,420.72,418.9851,419.18,58438340,0.000334,0.005757,0.009150
2021-05-28 04:00:00+00:00,420.97,421.25,419.7900,420.19,59768917,0.002409,0.008006,0.008014
2021-06-01 04:00:00+00:00,422.57,422.72,419.2000,419.67,55475267,-0.001238,0.007332,0.010892
2021-06-02 04:00:00+00:00,420.37,421.23,419.2900,420.39,50203715,0.001716,0.017064,0.014434
2021-06-03 04:00:00+00:00,417.85,419.99,416.2800,418.83,59499680,-0.003711,0.005711,0.007589


In [5]:
# Set the Signal column
signals_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
signals_df["Signal"][short_window:] = np.where(
    signals_df["Short"][short_window:] > signals_df["Long"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df.iloc[95:105, :]

C:\Users\DOUGHJ-WS1\AppData\Local\Temp\ipykernel_3560\4127829247.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals_df["Signal"][short_window:] = np.where(


,open,high,low,close,volume,Actual Returns,Short,Long,Signal,Entry/Exit
timestamp,,,,,,,,,,
2021-05-21 04:00:00+00:00,416.87,418.20,414.4500,414.98,77590498,-0.000409,0.001745,0.009461,0.0,0.0
2021-05-24 04:00:00+00:00,417.34,420.32,417.0800,419.17,52606963,0.010097,0.006436,0.013218,0.0,0.0
2021-05-25 04:00:00+00:00,420.33,420.71,417.6200,418.33,58365218,-0.002004,0.008189,0.012250,0.0,0.0
2021-05-26 04:00:00+00:00,418.87,419.61,417.7600,419.04,43861538,0.001697,0.005685,0.011426,0.0,0.0
2021-05-27 04:00:00+00:00,420.17,420.72,418.9851,419.18,58438340,0.000334,0.005757,0.009150,0.0,0.0
2021-05-28 04:00:00+00:00,420.97,421.25,419.7900,420.19,59768917,0.002409,0.008006,0.008014,0.0,0.0
2021-06-01 04:00:00+00:00,422.57,422.72,419.2000,419.67,55475267,-0.001238,0.007332,0.010892,0.0,0.0
2021-06-02 04:00:00+00:00,420.37,421.23,419.2900,420.39,50203715,0.001716,0.017064,0.014434,1.0,1.0
2021-06-03 04:00:00+00:00,417.85,419.99,416.2800,418.83,59499680,-0.003711,0.005711,0.007589,0.0,-1.0


In [6]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["Short", "Long"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot

:Overlay
   .Curve.I          :Curve   [timestamp]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

### Step 3: Split the data into training and testing datasets.

In [7]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['Short', 'Long']].shift().dropna()

# Review the DataFrame
X.head()

,Short,Long
timestamp,,
2021-02-19 05:00:00+00:00,0.027024,0.046552
2021-02-22 05:00:00+00:00,0.026882,0.041374
2021-02-23 05:00:00+00:00,0.023545,0.028347
2021-02-24 05:00:00+00:00,0.014921,0.015874
2021-02-25 05:00:00+00:00,0.026601,0.029297


In [8]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Signal']

# Review the value counts
y.value_counts()

0.0    404
1.0    349
Name: Signal, dtype: int64

In [9]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2021-02-19 05:00:00+00:00


In [10]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=6)

# Display the training end date
print(training_end)

2021-08-19 05:00:00+00:00


In [11]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

,Short,Long
timestamp,,
2021-02-19 05:00:00+00:00,0.027024,0.046552
2021-02-22 05:00:00+00:00,0.026882,0.041374
2021-02-23 05:00:00+00:00,0.023545,0.028347
2021-02-24 05:00:00+00:00,0.014921,0.015874
2021-02-25 05:00:00+00:00,0.026601,0.029297


In [12]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_train.head()

,Short,Long
timestamp,,
2021-02-19 05:00:00+00:00,0.027024,0.046552
2021-02-22 05:00:00+00:00,0.026882,0.041374
2021-02-23 05:00:00+00:00,0.023545,0.028347
2021-02-24 05:00:00+00:00,0.014921,0.015874
2021-02-25 05:00:00+00:00,0.026601,0.029297


In [13]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Step 4: Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

In [14]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred


array([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0.

### Step 5: Review the classification report associated with the `SVC` model predictions. 

In [15]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)


              precision    recall  f1-score   support

         0.0       0.58      0.96      0.72       303
         1.0       0.86      0.27      0.42       292

    accuracy                           0.62       595
   macro avg       0.72      0.62      0.57       595
weighted avg       0.72      0.62      0.57       595



### Step 6: Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

In [16]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = (predictions_df['Actual Returns'] * predictions_df['Predicted'])

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

,Predicted,Actual Returns,Strategy Returns
timestamp,,,
2021-08-20 04:00:00+00:00,0.0,0.007957,0.000000
2021-08-23 04:00:00+00:00,0.0,0.008796,0.000000
2021-08-24 04:00:00+00:00,0.0,0.001587,0.000000
2021-08-25 04:00:00+00:00,1.0,0.002098,0.002098
2021-08-26 04:00:00+00:00,1.0,-0.005903,-0.005903


,Predicted,Actual Returns,Strategy Returns
timestamp,,,
2023-12-26 05:00:00+00:00,0.0,0.004223,0.0
2023-12-27 05:00:00+00:00,0.0,0.001808,0.0
2023-12-28 05:00:00+00:00,0.0,0.000378,0.0
2023-12-29 05:00:00+00:00,0.0,-0.002895,-0.0
2024-01-02 05:00:00+00:00,0.0,-0.005596,-0.0


### Step 7: Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

In [17]:
# Plot the actual returns versus the strategy returns
(1 + predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot()


:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

### Step 1:  Choose a new classifier to test with.

In [18]:
# Import a new classifier from SKLearn
from sklearn.linear_model import LogisticRegressionCV

# Initiate the model instance
new_model = LogisticRegressionCV()


### Step 2: Using the original training data as the baseline model, fit another model with the new classifier.

In [19]:
# Fit the model using the training data
new_model = new_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
new_pred = new_model.predict(X_test_scaled)

# Review the model's predicted values
new_pred


array([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1.

In [20]:
# Use a classification report to evaluate the model using the predictions and testing data
new_testing_report = classification_report(y_test, new_pred)

# Print the classification report
print(new_testing_report)


              precision    recall  f1-score   support

         0.0       0.88      0.89      0.88       303
         1.0       0.88      0.87      0.88       292

    accuracy                           0.88       595
   macro avg       0.88      0.88      0.88       595
weighted avg       0.88      0.88      0.88       595



In [21]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
new_pred_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
new_pred_df['New Predicted'] = new_pred

# Add the actual returns to the DataFrame
new_pred_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
new_pred_df['New Strategy Returns'] = (new_pred_df['Actual Returns'] * new_pred_df['New Predicted'])

# Review the DataFrame
display(new_pred_df.head())
display(new_pred_df.tail())


,New Predicted,Actual Returns,New Strategy Returns
timestamp,,,
2021-08-20 04:00:00+00:00,0.0,0.007957,0.000000
2021-08-23 04:00:00+00:00,0.0,0.008796,0.000000
2021-08-24 04:00:00+00:00,0.0,0.001587,0.000000
2021-08-25 04:00:00+00:00,1.0,0.002098,0.002098
2021-08-26 04:00:00+00:00,1.0,-0.005903,-0.005903


,New Predicted,Actual Returns,New Strategy Returns
timestamp,,,
2023-12-26 05:00:00+00:00,0.0,0.004223,0.000000
2023-12-27 05:00:00+00:00,0.0,0.001808,0.000000
2023-12-28 05:00:00+00:00,0.0,0.000378,0.000000
2023-12-29 05:00:00+00:00,0.0,-0.002895,-0.000000
2024-01-02 05:00:00+00:00,1.0,-0.005596,-0.005596


In [22]:
# Plot the actual returns versus the strategy returns
(1 + new_pred_df[['Actual Returns', 'New Strategy Returns']]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [23]:
# Create a list for the column name
columns = ["Actual"]

# Create a list holding the names of the new evaluation metrics
metrics = [
    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio"]

# Initialize the DataFrame with index set to the evaluation metrics and the column
spy_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

In [24]:
 # Calculate annualized return
spy_evaluation_df.loc["Annualized Return"] = (
    new_pred_df["Actual Returns"].mean() * 252
)

# Calculate cumulative return
spy_evaluation_df.loc["Cumulative Returns"] = (1 + new_pred_df["Actual Returns"]).cumprod()[-1]

 # Calculate annual volatility
spy_evaluation_df.loc["Annual Volatility"] = (
    new_pred_df["Actual Returns"].std() * np.sqrt(252)
)

# Calculate Sharpe ratio
spy_evaluation_df.loc["Sharpe Ratio"] = (
    new_pred_df["Actual Returns"].mean() * 252) / (
    new_pred_df["Actual Returns"].std() * np.sqrt(252)
)

# Review the result
spy_evaluation_df

,Actual
Annualized Return,0.047991
Cumulative Returns,1.074546
Annual Volatility,0.187369
Sharpe Ratio,0.256132


In [25]:
# Create a list for the column name
columns = ["Strategy"]

# Create a list holding the names of the new evaluation metrics
metrics = [
    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio"]

# Initialize the DataFrame with index set to the evaluation metrics and the column
strategy_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

In [26]:
 # Calculate annualized return
strategy_evaluation_df.loc["Annualized Return"] = (
    new_pred_df["New Strategy Returns"].mean() * 252
)

# Calculate cumulative return
strategy_evaluation_df.loc["Cumulative Returns"] = (1 + new_pred_df["New Strategy Returns"]).cumprod()[-1]

 # Calculate annual volatility
strategy_evaluation_df.loc["Annual Volatility"] = (
    new_pred_df["New Strategy Returns"].std() * np.sqrt(252)
)

# Calculate Sharpe ratio
strategy_evaluation_df.loc["Sharpe Ratio"] = (
    new_pred_df["New Strategy Returns"].mean() * 252) / (
    new_pred_df["New Strategy Returns"].std() * np.sqrt(252)
)

# Review the result
strategy_evaluation_df

,Strategy
Annualized Return,0.109651
Cumulative Returns,1.270151
Annual Volatility,0.129516
Sharpe Ratio,0.846619


In [27]:
pd.concat([spy_evaluation_df, strategy_evaluation_df], axis=1)

,Actual,Strategy
Annualized Return,0.047991,0.109651
Cumulative Returns,1.074546,1.270151
Annual Volatility,0.187369,0.129516
Sharpe Ratio,0.256132,0.846619
